In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D
model=Sequential()
model.add(Conv1D(filters=16,
                 kernel_size=3,
                 padding='same', # same can be replaced by valid as they are the same options
                 activation='relu',
                 strides=1,
                 input_shape=(100,300))) # you don't change the input shape the padding happens under the hood

In [ ]:
# Building the CNN froms scratch while using custom raw data
import numpy as np
from keras.preprocessing import sequence # helps in padding input
from keras.models import Sequential # base keras model
from keras.layers import Dense,Dropout,Activation
from keras.layers import Conv1D, GlobalMaxPooling1D # Convolution layer and poolingg


In [ ]:
# we will use the imdb dataset
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [ ]:
!pip install nlpia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.6/706.6 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.8 MB/s eta 0:00:00


In [ ]:
from datasets import list_datasets
from datasets import load_dataset
imdb=load_dataset("imdb")
imdb

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
imdb["test"][2500]["label"]


0

In [ ]:
# tokenization
import gensim.downloader as api
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
w2v_model=api.load("word2vec-google-news-300")
w2v_model.save_word2vec_format('word2vec-google-news-300.bin',binary=True)
# make sure you save the model by its name to enable it save properly



[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
def tokenize_and_vectorize(dataset, word_vectors):
    """Tokenizes text data using TreebankWordTokenizer and vectorizes using provided word vectors.

    Args:
        dataset: Dataset containing text data, where each sample is a list/tuple of (text, label).
        word_vectors: Loaded Word2Vec model for vectorization.

    Returns:
        List of lists, where each inner list represents tokenized and vectorized text.
    """

    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected=[]

    for sample in dataset:
        text = sample[0]  # Access text from the second element of the sample
        tokens = tokenizer.tokenize(text)
        sample_vecs = []

        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except KeyError:
                # Handle missing tokens:
                sample_vecs.append(np.zeros(word_vectors.vector_size))  # Pad with zeros

        vectorized_data.append(sample_vecs)

    return vectorized_data


In [ ]:
import torch
w2v_model = KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)

# Convert to PyTorch tensor
word_vectors = torch.tensor(w2v_model.vectors)

# Move the tensor to the GPU
word_vectors = word_vectors.to('cuda')

In [ ]:
import datasets

# Get label names directly from the feature:
label_names = imdb["train"].features["label"].names

# Create the encoder:
encoder = datasets.ClassLabel(num_classes=len(label_names), names=label_names)

# Apply encoding to each split:
for split in imdb.keys():
    imdb[split] = imdb[split].map(
        lambda example: {"label": encoder.str2int(example["label"])},  # Use str2int method for encoding
        batched=True,
    )

# Access encoded labels:
encoded_label = imdb["train"][0]["label"]


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
vectorized_training = tokenize_and_vectorize(imdb["train"]["text"],word_vectors)

In [ ]:
vectorized_testing=tokenize_and_vectorize(imdb["test"]["text"],word_vectors)

In [ ]:
print(imdb["train"].features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [ ]:
# getting the labels
def collect_expected(dataset):
    """ Peel off the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample['label'])  # Access the 'label' column directly
    return expected

expected_training = collect_expected(imdb["train"])

In [ ]:
expected_testing=collect_expected(imdb["test"])

In [ ]:
def pad_trunc(data,maxlen):
  """ In the dataset pad with zero vectors or truncate to maxlen """
  new_data=[]
  # create a vector of 0s the length of our word vectors
  zero_vector=[]
  for _ in range(len(data[0][0])):
    zero_vector.append(0.0)
  for sample in data:
    if len(sample) > maxlen:
      temp = sample[:maxlen]
    elif len(sample) < maxlen:
      temp =sample
      # Append the appropriate number 0 vectors to the list
      additional_elements= maxlen- len(sample)
      for _ in range(additional_elements):
        temp.append(zero_vector)
    else:
      temp=sample
    new_data.append(temp)
  return new_data

In [ ]:
# define the CNN parameters
maxlen = 400
batch_size = 32
embedding_dims = 300
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [ ]:
X_train=pad_trunc(vectorized_training ,maxlen)
X_test=pad_trunc(vectorized_testing ,maxlen)

x_train= np.reshape(X_train, (len(X_train), maxlen, embedding_dims))
y_train = np.array(expected_training)

x_test= np.reshape(X_test, (len(X_test), maxlen, embedding_dims))
y_test = np.array(expected_testing)


NameError: name 'pad_trunc' is not defined

In [ ]:
# Building the neural network a 1D CNN
print('Build model...')
model=Sequential()
model.add(Conv1D(
    filters,
    kernel_sizem
    padding='valid',
    activation='relu',
    strides=1,
    input_shape=(maxlen,embedding_dims)
))

In [ ]:
# add max pooling to save on computational costs, enable dimension reduction and ensure location invariance
model.add(GlobalMaxPooling1D())